# Download data

In [1]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ifood-2019-fgvc6

Saving kaggle.json to kaggle.json
kaggle.json
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 82.8MB/s]
  0% 0.00/3.63k [00:00<?, ?B/s]
100% 3.63k/3.63k [00:00<00:00, 3.44MB/s]
  0% 0.00/217k [00:00<?, ?B/s]
100% 217k/217k [00:00<00:00, 68.0MB/s]
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 76.0MB/s]
 96% 505M/526M [00:04<00:00, 131MB/s]
100% 526M/526M [00:04<00:00, 118MB/s]
 92% 204M/221M [00:02<00:00, 108MB/s]
100% 221M/221M [00:02<00:00, 110MB/s]
100% 2.11G/2.12G [00:48<00:00, 163MB/s]
100% 2.12G/2.12G [00:48<00:00, 47.0MB/s]
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 185MB/s]


In [2]:
!mkdir data
!unzip train_set.zip -d data
!unzip val_set.zip -d data
!unzip test_set.zip -d data
!unzip train_labels.csv.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/test_set/test_009481.jpg  
  inflating: data/test_set/test_013960.jpg  
  inflating: data/test_set/test_002362.jpg  
  inflating: data/test_set/test_005803.jpg  
  inflating: data/test_set/test_015388.jpg  
  inflating: data/test_set/test_002764.jpg  
  inflating: data/test_set/test_021964.jpg  
  inflating: data/test_set/test_013738.jpg  
  inflating: data/test_set/test_016448.jpg  
  inflating: data/test_set/test_022310.jpg  
  inflating: data/test_set/test_002203.jpg  
  inflating: data/test_set/test_003963.jpg  
  inflating: data/test_set/test_022431.jpg  
  inflating: data/test_set/test_003303.jpg  
  inflating: data/test_set/test_006227.jpg  
  inflating: data/test_set/test_016860.jpg  
  inflating: data/test_set/test_009457.jpg  
  inflating: data/test_set/test_020554.jpg  
  inflating: data/test_set/test_003655.jpg  
  inflating: data/test_set/test_006939.jpg  
  inflating: data/test_set/test_003241.jpg  
  in

#ResNet152 Model

## import libaries

In [0]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import random
random.seed(1)

## Data preparation

In [0]:
# create class to label
def createclass2label(path):
    table = pd.read_table(path, header=None, sep=' ')
    table.columns = ['label', 'class']
    class_dic = {}
    n = len(table)
    for i in range(n):
        class_dic[table.iloc[i]['class']] = table.iloc[i]['label']
    return class_dic

In [0]:
class_list_path = os.path.join("class_list.txt")
class FoodDataset(Dataset):
    def __init__(self, data_dir, data_df, transform=None, class_list_dir=class_list_path):
        self.food_label = createclass2label(class_list_dir)
        self.data_dir = data_dir
        self.data_info = data_df
        self.transform = transform

    def __getitem__(self, index):
        path_img = os.path.join(self.data_dir, self.data_info.iloc[index]['img_name'])
        label = self.data_info.iloc[index]['label']
        img = Image.open(path_img).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data_info)

In [0]:
train_label_dir = os.path.join("data", "train_labels.csv")
valid_label_dir = os.path.join("val_labels.csv")
test_label_dir = os.path.join("sample_submission.csv")

In [0]:
train_df = pd.read_csv(train_label_dir)
valid_df = pd.read_csv(valid_label_dir)

In [0]:
MAX_EPOCH = 10
BATCH_SIZE = 20
LR = 0.001
log_interval = 10
val_interval = 1
N_CLASSES = 251
TRAIN_SAMPLE = 100000
VALID_SAMPLE = 10000
checkpoint_interval = 1

In [0]:
train_data_df = train_df.sample(TRAIN_SAMPLE)
valid_data_df = valid_df.sample(VALID_SAMPLE)
test_data_df = pd.read_csv(test_label_dir)

In [10]:
print(f'Found {train_data_df.shape[0]} train images in {len(train_data_df.label.unique())} classes')
print(f'Found {valid_data_df.shape[0]} valid images in {len(valid_data_df.label.unique())} classes')
print(f'Found {test_data_df.shape[0]} test images in {len(test_data_df.label.unique())} classes')

Found 100000 train images in 251 classes
Found 10000 valid images in 251 classes
Found 28377 test images in 1 classes


In [0]:
data_dir = os.path.join("data")
train_dir = os.path.join(data_dir, "train_set")
valid_dir = os.path.join(data_dir, "val_set")
test_dir = os.path.join(data_dir, "test_set")

In [0]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

train_data = FoodDataset(data_dir=train_dir, data_df=train_data_df, transform=train_transform)
valid_data = FoodDataset(data_dir=valid_dir, data_df=valid_data_df, transform=valid_transform)
test_data = FoodDataset(data_dir=test_dir, data_df=test_data_df, transform=test_transform)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

## Select model

In [13]:
resnet152_model = models.resnet152(pretrained=True)
# change fc layer
num_features = resnet152_model.fc.in_features

resnet152_model.fc = nn.Linear(num_features, N_CLASSES)
resnet152_model

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Loss Function

In [0]:
# Loss Function
# criterion = nn.NLLLoss()
criterion = nn.MultiMarginLoss()

## Choose optimizer

In [0]:
fc_params_id = list(map(id, resnet152_model.fc.parameters()))
base_params = filter(lambda p: id(p) not in fc_params_id, resnet152_model.parameters())
optimizer = optim.SGD([
    {'params': base_params, 'lr': LR * 0.1},    # 0
    {'params': resnet152_model.fc.parameters(), 'lr': LR}], momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

In [0]:
def train_classifier(model, print_every):
    train_curve = list()
    valid_curve = list()
    model.to('cuda')
    start_epoch = -1
    for e in range(start_epoch + 1, MAX_EPOCH):
        loss_mean = 0.
        correct = 0.
        total = 0.

        model.train()
        for i, data in enumerate(train_loader):

            # forward
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            output = model.forward(images)

            # backward
            optimizer.zero_grad()
            loss = criterion(output, labels)
            loss.backward()

            # update weights
            optimizer.step()

            # calculate loss and training infomation
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).squeeze().sum().cpu().numpy()

            # print
            loss_mean += loss.item()
            train_curve.append(loss.item())
            if (i + 1) % print_every == 0:
                loss_mean = loss_mean / print_every
                print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, i + 1, len(train_loader), loss_mean, correct / total))
                loss_mean = 0.

        scheduler.step()

        if (e + 1) % checkpoint_interval == 0:

            checkpoint = {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": e
            }
            path_checkpoint = f'./drive/My Drive/COMP540/checkpoints/resnet152/checkpoint_{e}_epoch.pkl'
            torch.save(checkpoint, path_checkpoint)

        # validation
        if (e + 1) % val_interval == 0:

            correct_val = 0.
            total_val = 0.
            loss_val = 0.
            model.eval()
            with torch.no_grad():
                for j, data in enumerate(valid_loader):
                    images, labels = data
                    images, labels = images.to('cuda'), labels.to('cuda')

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).squeeze().sum().cpu().numpy()

                    loss_val += loss.item()

                loss_val_mean = loss_val / len(valid_loader)
                valid_curve.append(loss.item())
                print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, j + 1, len(valid_loader), loss_val_mean, correct_val / total_val))
            model.train()

In [0]:
train_classifier(resnet152_model, print_every=50)

Training:Epoch[000/010] Iteration[050/3125] Loss: 1.0079 Acc:0.00%
Training:Epoch[000/010] Iteration[100/3125] Loss: 0.9424 Acc:0.19%
Training:Epoch[000/010] Iteration[150/3125] Loss: 0.9356 Acc:0.38%
Training:Epoch[000/010] Iteration[200/3125] Loss: 0.8928 Acc:0.44%
Training:Epoch[000/010] Iteration[250/3125] Loss: 0.8646 Acc:0.65%
Training:Epoch[000/010] Iteration[300/3125] Loss: 0.8532 Acc:0.71%
Training:Epoch[000/010] Iteration[350/3125] Loss: 0.8129 Acc:0.93%
Training:Epoch[000/010] Iteration[400/3125] Loss: 0.7430 Acc:1.17%
Training:Epoch[000/010] Iteration[450/3125] Loss: 0.7445 Acc:1.35%
Training:Epoch[000/010] Iteration[500/3125] Loss: 0.7064 Acc:1.55%
Training:Epoch[000/010] Iteration[550/3125] Loss: 0.6836 Acc:1.77%
Training:Epoch[000/010] Iteration[600/3125] Loss: 0.6667 Acc:1.90%
Training:Epoch[000/010] Iteration[650/3125] Loss: 0.6452 Acc:2.09%
Training:Epoch[000/010] Iteration[700/3125] Loss: 0.6318 Acc:2.27%
Training:Epoch[000/010] Iteration[750/3125] Loss: 0.5815 Acc:2

In [0]:
def save_model(model, name, save_state_dic=False):
    model_path = f'./drive/My Drive/COMP540/{name}.pkl'
    if save_state_dic:
        path_state_dict = f'./drive/My Drive/COMP540/{name}_state_dict.pkl'
        model_state_dict = model.state_dict()
        torch.save(model_state_dict, path_state_dict)
    torch.save(model, model_path)

In [0]:
def load_model(model_path, state_dic_path=None):

    model_load = torch.load(model_path)
    if state_dic_path is not None:
        state_dict_load = torch.load(state_dic_path)
        return model_load, state_dict_load
    return model_load

## Resume training

In [0]:
def train_classifier_resume(model, optimizer, path_checkpoint, print_every = 50):
    checkpoint = torch.load(path_checkpoint)
    model.to('cuda')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    train_curve = list()
    valid_curve = list()
    for e in range(start_epoch + 1, MAX_EPOCH):

        loss_mean = 0.
        correct = 0.
        total = 0.

        model.train()
        for i, data in enumerate(train_loader):

            # forward
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            output = model.forward(images)

            # backward
            optimizer.zero_grad()
            loss = criterion(output, labels)
            loss.backward()

            # update weights
            optimizer.step()

            # calculate loss and training infomation
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).squeeze().sum().cpu().numpy()

            # print
            loss_mean += loss.item()
            train_curve.append(loss.item())
            if (i + 1) % print_every == 0:
                loss_mean = loss_mean / print_every
                print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, i + 1, len(train_loader), loss_mean, correct / total))
                loss_mean = 0.

        scheduler.step()

        if (e + 1) % checkpoint_interval == 0:
            checkpoint = {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": e
            }
            path_checkpoint = f'./drive/My Drive/COMP540/checkpoints/resnet152/checkpoint_{e}_epoch.pkl'
            torch.save(checkpoint, path_checkpoint)

        # validation
        if (e + 1) % val_interval == 0:

            correct_val = 0.
            total_val = 0.
            loss_val = 0.
            model.eval()
            with torch.no_grad():
                for j, data in enumerate(valid_loader):
                    images, labels = data
                    images, labels = images.to('cuda'), labels.to('cuda')

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).squeeze().sum().cpu().numpy()

                    loss_val += loss.item()

                loss_val_mean = loss_val / len(valid_loader)
                valid_curve.append(loss.item())
                print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, j + 1, len(valid_loader), loss_val_mean, correct_val / total_val))
            model.train()

In [0]:
path_checkpoint = './drive/My Drive/COMP540/checkpoints/resnet152/checkpoint_9_epoch.pkl'
train_classifier_resume(resnet152_model, optimizer, path_checkpoint, print_every=100)

In [0]:
name = 'resnet152'
save_model(resnet152_model, name, save_state_dic=True)

In [22]:
train_data.food_label 

{'adobo': 47,
 'ambrosia_food': 129,
 'apple_pie': 29,
 'apple_turnover': 244,
 'applesauce': 72,
 'applesauce_cake': 237,
 'baby_back_rib': 206,
 'bacon_and_eggs': 118,
 'bacon_lettuce_tomato_sandwich': 232,
 'baked_alaska': 173,
 'baklava': 73,
 'barbecued_spareribs': 240,
 'barbecued_wing': 215,
 'beef_bourguignonne': 241,
 'beef_carpaccio': 114,
 'beef_stroganoff': 41,
 'beef_tartare': 18,
 'beef_wellington': 42,
 'beet_salad': 82,
 'beignet': 1,
 'bibimbap': 27,
 'biryani': 177,
 'blancmange': 149,
 'boiled_egg': 124,
 'boston_cream_pie': 225,
 'bread_pudding': 169,
 'brisket': 126,
 'bruschetta': 150,
 'bubble_and_squeak': 58,
 'buffalo_wing': 249,
 'burrito': 198,
 'caesar_salad': 94,
 'cannelloni': 182,
 'cannoli': 19,
 'caprese_salad': 60,
 'carbonnade_flamande': 231,
 'carrot_cake': 52,
 'casserole': 147,
 'ceviche': 141,
 'cheesecake': 188,
 'chicken_cordon_bleu': 246,
 'chicken_curry': 92,
 'chicken_kiev': 28,
 'chicken_marengo': 222,
 'chicken_provencale': 228,
 'chicken_q

In [0]:
label_to_class = {v : u for u, v in train_data.food_label.items()}

In [25]:
label_to_class

{0: 'macaron',
 1: 'beignet',
 2: 'cruller',
 3: 'cockle_food',
 4: 'samosa',
 5: 'tiramisu',
 6: 'tostada',
 7: 'moussaka',
 8: 'dumpling',
 9: 'sashimi',
 10: 'knish',
 11: 'croquette',
 12: 'couscous',
 13: 'porridge',
 14: 'stuffed_cabbage',
 15: 'seaweed_salad',
 16: 'chow_mein',
 17: 'rigatoni',
 18: 'beef_tartare',
 19: 'cannoli',
 20: 'foie_gras',
 21: 'cupcake',
 22: 'osso_buco',
 23: 'pad_thai',
 24: 'poutine',
 25: 'ramen',
 26: 'pulled_pork_sandwich',
 27: 'bibimbap',
 28: 'chicken_kiev',
 29: 'apple_pie',
 30: 'risotto',
 31: 'fruitcake',
 32: 'chop_suey',
 33: 'haggis',
 34: 'scrambled_eggs',
 35: 'frittata',
 36: 'scampi',
 37: 'sushi',
 38: 'orzo',
 39: 'fritter',
 40: 'nacho',
 41: 'beef_stroganoff',
 42: 'beef_wellington',
 43: 'spring_roll',
 44: 'savarin',
 45: 'crayfish_food',
 46: 'souffle',
 47: 'adobo',
 48: 'streusel',
 49: 'deviled_egg',
 50: 'escargot',
 51: 'club_sandwich',
 52: 'carrot_cake',
 53: 'falafel',
 54: 'farfalle',
 55: 'terrine',
 56: 'poached_eg

## Inference

In [0]:
model_path = "./drive/My Drive/COMP540/modelsaves/resnet152/resnet152_state_dict.pkl"

In [32]:
state_dict = torch.load(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model = models.resnet152()
num_features = inference_model.fc.in_features
inference_model.fc = nn.Linear(num_features, N_CLASSES)
inference_model.load_state_dict(state_dict)
inference_model.to(device)
inference_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [35]:
pred_list = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, _ = data
        images = images.to(device)

        # tensor to vector
        outputs = inference_model(images)

        # convert to df
        _, pred_int = torch.max(outputs.data, 1)
        pred_list.append(pred_int.cpu().numpy().reshape((1, -1)).tolist())
        print(f'Finished {i+1} prediction')

Finished 1 prediction
Finished 2 prediction
Finished 3 prediction
Finished 4 prediction
Finished 5 prediction
Finished 6 prediction
Finished 7 prediction
Finished 8 prediction
Finished 9 prediction
Finished 10 prediction
Finished 11 prediction
Finished 12 prediction
Finished 13 prediction
Finished 14 prediction
Finished 15 prediction
Finished 16 prediction
Finished 17 prediction
Finished 18 prediction
Finished 19 prediction
Finished 20 prediction
Finished 21 prediction
Finished 22 prediction
Finished 23 prediction
Finished 24 prediction
Finished 25 prediction
Finished 26 prediction
Finished 27 prediction
Finished 28 prediction
Finished 29 prediction
Finished 30 prediction
Finished 31 prediction
Finished 32 prediction
Finished 33 prediction
Finished 34 prediction
Finished 35 prediction
Finished 36 prediction
Finished 37 prediction
Finished 38 prediction
Finished 39 prediction
Finished 40 prediction
Finished 41 prediction
Finished 42 prediction
Finished 43 prediction
Finished 44 predicti

ValueError: ignored

In [0]:
new_pred_list = []
for i in range(len(pred_list)):
    for j in range(len(pred_list[i][0])):
        new_pred_list.append(pred_list[i][0][j])
test_data_df['pred_label'] = new_pred_list
test_data_df.to_csv('./drive/My Drive/COMP540/resnet152.csv')

In [59]:
test_data_df

,img_name,label,pred_label
0,test_024088.jpg,250,212
1,test_024089.jpg,250,232
2,test_024090.jpg,250,197
3,test_024091.jpg,250,236
4,test_024092.jpg,250,231
...,...,...,...
28372,test_024083.jpg,250,169
28373,test_024084.jpg,250,163
28374,test_024085.jpg,250,184
28375,test_024086.jpg,250,88
